In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
df=pd.read_csv('/content/fakenewsdataset.zip')

In [3]:
df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [4]:
!pip install plotly
!pip install nltk
!pip install spacy 
!pip install gensim 
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [7]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model

In [8]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [10]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [11]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [12]:
df['clean'] = df['text'].apply(preprocess)

In [13]:
list_of_words = []
for i in df['clean']:
    for j in i:
        list_of_words.append(j)

In [14]:
len(list_of_words)

2304425

In [15]:
total_words = len(list(set(list_of_words)))
total_words

62148

In [16]:
df

,Unnamed: 0,title,text,label,clean
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,"[daniel, greenfield, shillman, journalism, fel..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,"[google, pinterest, digg, linkedin, reddit, st..."
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,"[secretary, state, john, kerry, said, monday, ..."
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,"[kaydee, king, kaydeeking, november, lesson, t..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,"[primary, york, runners, hillary, clinton, don..."
...,...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL,"[state, department, told, republican, national..."
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE,"[stand, plutocratic, pentagon, posted, wikimed..."
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE,"[anti, trump, protesters, tools, oligarchy, re..."
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL,"[addis, ababa, ethiopia, president, obama, con..."


In [17]:
df['clean_joined'] = df['clean'].apply(lambda x: " ".join(x))

In [18]:
df

,Unnamed: 0,title,text,label,clean,clean_joined
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,"[daniel, greenfield, shillman, journalism, fel...",daniel greenfield shillman journalism fellow f...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,"[google, pinterest, digg, linkedin, reddit, st...",google pinterest digg linkedin reddit stumbleu...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,"[secretary, state, john, kerry, said, monday, ...",secretary state john kerry said monday stop pa...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,"[kaydee, king, kaydeeking, november, lesson, t...",kaydee king kaydeeking november lesson tonight...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,"[primary, york, runners, hillary, clinton, don...",primary york runners hillary clinton donald tr...
...,...,...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL,"[state, department, told, republican, national...",state department told republican national comm...
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE,"[stand, plutocratic, pentagon, posted, wikimed...",stand plutocratic pentagon posted wikimedia te...
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE,"[anti, trump, protesters, tools, oligarchy, re...",anti trump protesters tools oligarchy reform p...
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL,"[addis, ababa, ethiopia, president, obama, con...",addis ababa ethiopia president obama convened ...


In [19]:
maxlen = -1
for doc in df.clean_joined:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print( maxlen)

7967


In [20]:
df = pd.get_dummies(df, columns=['label'], drop_first=True)

In [21]:
df

,Unnamed: 0,title,text,clean,clean_joined,label_REAL
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...","[daniel, greenfield, shillman, journalism, fel...",daniel greenfield shillman journalism fellow f...,0
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,"[google, pinterest, digg, linkedin, reddit, st...",google pinterest digg linkedin reddit stumbleu...,0
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,"[secretary, state, john, kerry, said, monday, ...",secretary state john kerry said monday stop pa...,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...","[kaydee, king, kaydeeking, november, lesson, t...",kaydee king kaydeeking november lesson tonight...,0
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,"[primary, york, runners, hillary, clinton, don...",primary york runners hillary clinton donald tr...,1
...,...,...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,"[state, department, told, republican, national...",state department told republican national comm...,1
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,"[stand, plutocratic, pentagon, posted, wikimed...",stand plutocratic pentagon posted wikimedia te...,0
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,"[anti, trump, protesters, tools, oligarchy, re...",anti trump protesters tools oligarchy reform p...,0
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...","[addis, ababa, ethiopia, president, obama, con...",addis ababa ethiopia president obama convened ...,1


In [22]:
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(df['clean_joined'].values)
x = tokenizer.texts_to_sequences(df['clean_joined'])

x = pad_sequences(x, maxlen=80)
y = df['label_REAL']

In [23]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=2021,test_size=0.1)


In [24]:
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Embedding, LSTM

In [25]:
rnn_model=Sequential()

In [26]:
rnn_model.add(Embedding(500, 64, input_length=80))

rnn_model.add(LSTM(192, dropout=0.2, recurrent_dropout=0.2, return_sequences = True))

rnn_model.add(LSTM(96, dropout=0.2, recurrent_dropout=0.2))

rnn_model.add(Dense(1, activation='sigmoid'))

rnn_model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

In [27]:
rnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 80, 64)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 80, 192)           197376    
_________________________________________________________________
lstm_1 (LSTM)                (None, 96)                110976    
_________________________________________________________________
dense (Dense)                (None, 1)                 97        
Total params: 340,449
Trainable params: 340,449
Non-trainable params: 0
_________________________________________________________________


In [28]:
rnn_model.fit(
    x_train, y_train,
    validation_data = (x_test, y_test),
    epochs = 7,batch_size=32
)

Epoch 1/7
179/179 [==============================] - 101s 532ms/step - loss: 0.4853 - accuracy: 0.7732 - val_loss: 0.3825 - val_accuracy: 0.8407
Epoch 2/7
179/179 [==============================] - 95s 532ms/step - loss: 0.3581 - accuracy: 0.8486 - val_loss: 0.3360 - val_accuracy: 0.8628
Epoch 3/7
179/179 [==============================] - 95s 532ms/step - loss: 0.2949 - accuracy: 0.8770 - val_loss: 0.3288 - val_accuracy: 0.8628
Epoch 4/7
179/179 [==============================] - 96s 535ms/step - loss: 0.2830 - accuracy: 0.8839 - val_loss: 0.3249 - val_accuracy: 0.8785
Epoch 5/7
179/179 [==============================] - 95s 531ms/step - loss: 0.2544 - accuracy: 0.8997 - val_loss: 0.2919 - val_accuracy: 0.8912
Epoch 6/7
179/179 [==============================] - 96s 534ms/step - loss: 0.2473 - accuracy: 0.9019 - val_loss: 0.3008 - val_accuracy: 0.8864
Epoch 7/7
179/179 [==============================] - 96s 538ms/step - loss: 0.2217 - accuracy: 0.9116 - val_loss: 0.3064 - val_accuracy